In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [6]:
# Load the energy data from an Excel file, skipping unnecessary rows and footer

Energy = pd.read_excel('Energy Indicators.xls', skiprows=17, skipfooter=38)

# Drop the first two unnecessary columns

Energy=Energy.drop(columns=list(Energy.columns[0:2]))

# Rename the columns as per the required format

new=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
old=Energy.columns[:]
Energy=Energy.rename(columns=dict(zip(old,new)))

# Convert Energy Supply from petajoules to gigajoules (multiply by 1,000,000)

Energy['Energy Supply']=Energy['Energy Supply']*10000004

# Replace '...' values with NaN to mark missing data

Energy.replace("...",np.nan,inplace=True)

# Clean the 'Country' column by removing digits and special characters

Energy['Country'] = Energy['Country'].replace(to_replace=r'\d+|', value='', regex=True)

# Rename specific countries as required

rename={"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong",
       'The former Yugoslav Republic of Macedonia':'Macedonia'
       ,'Syrian Arab Republic':'Syrian','Turks and Caicos Islands':'Turky',
        "Democratic People's Republic of Korea":'Korea','Democratic Republic of the Congo':'Congo'
    }

old=list(rename.keys())
new=list(rename.values())
Energy['Country'].replace(to_replace=old,value=new,inplace=True)

# Further clean country names by removing parentheses and trailing spaces

Energy['Country'] = Energy['Country'].replace(to_replace=r'\(.*?\)' , value='', regex=True)
Energy['Country'] = Energy['Country'].replace(to_replace=r'\d+' , value='', regex=True)
Energy['Country'] = Energy['Country'].str.strip()

In [7]:
# Load the GDP data, clean it, and rename columns as needed

GDP=pd.read_csv('world_bank.csv',skiprows=4)
GDP=GDP.rename(columns={'Country Name':'Country'})

# Load the GDP data, clean it, and rename columns as needed

GDP['Country'] = GDP['Country'].replace(to_replace=r'\(.*?\)' , value='', regex=True)
Energy['Country'] = Energy['Country'].replace(to_replace=r'\d+' , value='', regex=True)
Energy['Country'] = Energy['Country'].str.strip()
rename={"Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong",'Korea':'South Korea'}
old=list(rename.keys())
new=list(rename.values())
GDP['Country'].replace(to_replace=old,value=new,inplace=True)
GDP=GDP.drop(columns=GDP.columns[1:50])


In [8]:
# Load Scimagojr data for country rankings

scimagojr=pd.read_excel('scimagojr-3.xlsx')


merge=pd.merge(scimagojr,Energy,on='Country',how='outer')
df=pd.merge(merge,GDP,on='Country',how='outer')
df=df.set_index('Country')
len(df['Rank'])-15

307

In [9]:
# Filter and sort data based on rank

df=df.query('Rank<16')
df=df.sort_values('Rank')
df.shape

(15, 20)

In [10]:
# Create column for renewable energy > average

round(np.mean(df['Energy Supply per Capita'],axis=0),2)

153.07

In [11]:
# Calculate statistics: max % renewable and related country

max_=np.max(df['% Renewable'])
index_=df.loc[df['% Renewable']==max_].index[0]
print(index_,' ',round(max_,2),'%')

Brazil   69.65 %


In [12]:
# Calculate self-citation percentage and find max country

df['Citations%']=round(df['Self-citations']/df['Citations']*100,2)
max_2=np.max(df['Citations%'])
index_2=df.loc[df['Citations%']==max_2].index[0]
print(index_2,' ',round(max_2,2),'%')

China   68.93 %


In [13]:
# Create column for renewable energy > average

mean_=np.mean(df['% Renewable'])
df['%Renewable>Avg']=(df['% Renewable']>mean_)*1
df['%Renewable>Avg']

Country
China                 0
United States         0
Japan                 0
United Kingdom        0
Russian Federation    0
Canada                1
Germany               0
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: %Renewable>Avg, dtype: int32

In [14]:
# Map countries to continents and create a new 'Area' column

ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
df['Area']=df.index.map(lambda x:ContinentDict.get(x,'N/A'))
df.head(5)


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2009,2010,2011,2012,2013,2014,2015,Citations%,%Renewable>Avg,Area
Country,,,,,,,,,,,,,,,,,,,,,
China,1.0,127050.0,126767.0,597237.0,411683.0,4.70,138.0,1271910508764,93.0,19.75491,...,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,68.93,0,Asia
United States,2.0,96661.0,94747.0,792274.0,265436.0,8.20,230.0,908380363352,286.0,11.57098,...,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,33.50,0,North America
Japan,3.0,30504.0,30287.0,223024.0,61554.0,7.31,134.0,189840075936,149.0,10.23282,...,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,27.60,0,Asia
United Kingdom,4.0,20944.0,20357.0,206091.0,37874.0,9.84,139.0,79200031680,124.0,10.60047,...,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,18.38,0,Europe
Russian Federation,5.0,18534.0,18301.0,34266.0,12422.0,1.85,57.0,307090122836,214.0,17.28868,...,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,36.25,0,Europe


In [15]:
df['population']=(df['Energy Supply']/df['Energy Supply per Capita'])
df['population']=df['population'].fillna(np.mean(df['population']))
(df['population']).astype(np.int64)

data=dict()
Area=dict(df['Area'].value_counts())
data['Area']=list(Area.keys())
data['Country_Number']=list(Area.values())
Area_sum=df.groupby('Area')['population'].sum()
Area_mean=df.groupby('Area')['population'].mean()
Area_std=df.groupby('Area')['population'].std()

In [16]:
final_data=pd.DataFrame(data)
final_data=final_data.merge(Area_sum,on='Area',how='inner')
final_data=final_data.merge(Area_mean,on='Area',how='inner')
final_data=final_data.merge(Area_std,on='Area',how='inner')
name=list(final_data.columns[2:])
names=dict(zip(name,['Total_population','AVG_population','std_population']))
final_data.rename(columns=names)

,Area,Country_Number,Total_population,AVG_population,std_population
0,Europe,6,4.579299e+09,7.632164e+08,3.464768e+08
1,Asia,5,3.126639e+10,6.253278e+09,6.412209e+09
2,North America,2,3.528554e+09,1.764277e+09,1.996697e+09
3,Australia,1,2.331603e+08,2.331603e+08,NaN
4,South America,1,2.059153e+09,2.059153e+09,NaN
